In [5]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import dbPass

import time

In [9]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie)
    altTitles = {}
    for key in ['Arabic', 'Cantonese', 'Chinese', 'French', 'Hangul', 'Hebrew', 'Hepburn', 'Japanese', 'Literally',  'Mandarin', 'McCune-Reischauer', 'Original title', 'Polish',  'Revised Romanization', 'Romanized', 'Russian', 'Simplified', 'Traditional', 'Yiddish']:
        if key in movie:
            altTitles[key] = movie[key]
            movie.pop(key)
    if len(altTitles) > 0:
        movie['alternate_titles'] = altTitles
        # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    return movie

In [13]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load(wiki_file, kaggle_file, ratings_file):
    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    ratings = pd.read_csv(ratings_file)
    wiki_movies_raw = []
    with open(wiki_file, mode='r') as file: 
        wiki_movies_raw = json.load(file)
        wiki_movies_raw = [movie for movie in wiki_movies_raw if ('Director' in movie or 'Directed by' in movie)
            and 'imdb_link' in movie
            and 'No. of episodes' not in movie]
        wiki_movies_raw = [clean_movie(movie) for movie in wiki_movies_raw] 
    wiki_movies_df = pd.DataFrame(wiki_movies_raw)

    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df["imdb_id"] = wiki_movies_df["imdb_link"].str.extract(r"(tt\d{7})")
        wiki_movies_df.drop_duplicates(subset="imdb_id", inplace=True)
    except:
        print("Something went wrong")
    #  Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
    # Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df["Box office"].dropna()

    
    # Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: " ".join(x) if type(x) == list else x)

    # Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r"\$\s*\d+\.?\d*\s*[mb]illi?on"
    # Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r"\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)"


    # Add the parse_dollars function.
    def parseDollars(string):
        if type(string) != str:
            return np.NaN
        # $###.# million and billion
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', string, flags=re.IGNORECASE):
            return float(re.sub('\$|\s|[a-zA-Z]','',string)) * 1000000
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', string, flags=re.IGNORECASE):
            return float(re.sub('\$|\s|[a-zA-Z]','',string)) * 1000000000
        #  $###,###,###.##
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', string, flags=re.IGNORECASE):
            return float(re.sub('\$|,','',string))
        else:
            return np.nan
        
    # Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df["box_office"] = box_office.str.extract(f"({form_one}|{form_two})", flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop("Box office", axis=1, inplace=True)
    
    # Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df["Budget"].dropna()
    budget = budget.map(lambda x: " ".join(x) if type(x) == list else x)
    budget = budget.str.replace(r"\$.*[-—–](?![a-z])", "$", regex=True)
    budget = budget.str.replace(r"\[\d+\]\s*", "")
    wiki_movies_df["budget"] = budget.str.extract(f"({form_one}|{form_two})", flags=re.IGNORECASE)[0].apply(parse_dollars)    

    # Clean the release date column in the wiki_movies_df DataFrame.
    wiki_movies_df.drop("Budget", axis=1, inplace=True)


    # Clean the running time column in the wiki_movies_df DataFrame.
    release_date = (wiki_movies_df["Release date"].dropna().apply(lambda x: " ".join(x) if type(x) == list else x))
    date_form_one = r"(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}"
    date_form_two = r"\d{4}.[01]\d.[123]\d"
    date_form_three = r"(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}"
    date_form_four = r"\d{4}"
    wiki_movies_df["release_date"] = pd.to_datetime(release_date.str.extract(f"({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})")[0],infer_datetime_format=True,)
     
    # 2. Clean the Kaggle metadata.

    # 3. Merged the two DataFrames into the movies DataFrame.


    # 4. Drop unnecessary columns from the merged DataFrame.


    # 5. Add in the function to fill in the missing Kaggle data.


    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.


    # 7. Filter the movies DataFrame for specific columns.


    # 8. Rename the columns in the movies DataFrame.


    # 9. Transform and merge the ratings DataFrame.
    
    return wiki_movies_df, movies_with_ratings_df, movies_df


In [7]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = 'resources'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [10]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load(wiki_file, kaggle_file, ratings_file)

NameError: name 'movies_with_ratings_df' is not defined

In [6]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [7]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

In [8]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

In [9]:
# 15. Check the movies_df DataFrame. 
movies_df.head()